In [1]:
import toolviper
import xradio
import pathlib
import numba

import numpy as np
import pandas as pd
import calviper as cv

from xradio import measurement_set as ms

In [2]:
if not pathlib.Path.cwd().joinpath("data/gaincaltest2.ps.zarr").exists():
    toolviper.utils.data.download("gaincal.test.zarr", "data")

In [3]:
ps = ms.open_processing_set("data/gaincaltest2.ps.zarr")

sub_ps = ps.sel(intents="CALIBRATE_DELAY#ON_SOURCE", scan_number=2)
sub_ps.summary()

,name,intents,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
1,gaincaltest2_0,"[CALIBRATE_DELAY#ON_SOURCE, CALIBRATE_PHASE#ON...","(957, 45, 8, 4)","[XX, XY, YX, YY]","[2, 4, 6, 9, 11, 14, 16, 18, 21, 23, 26]",X0000000000#ALMA_RB_03#BB_1#SW-01#FULL_RES_0,[J2255-3500_0],[J2255-3500_0],[],"[icrs, 22h55m57.68s, -35d00m00.00s]",8.607155e+10,8.629030e+10
0,gaincaltest2_2,"[CALIBRATE_DELAY#ON_SOURCE, CALIBRATE_PHASE#ON...","(957, 45, 8, 4)","[XX, XY, YX, YY]","[2, 4, 6, 9, 11, 14, 16, 18, 21, 23, 26]",X0000000000#ALMA_RB_03#BB_2#SW-01#FULL_RES_1,[J2255-3500_0],[J2255-3500_0],[],"[icrs, 22h55m57.68s, -35d00m00.00s]",8.794655e+10,8.816530e+10
3,gaincaltest2_4,"[CALIBRATE_DELAY#ON_SOURCE, CALIBRATE_PHASE#ON...","(957, 45, 8, 4)","[XX, XY, YX, YY]","[2, 4, 6, 9, 11, 14, 16, 18, 21, 23, 26]",X0000000000#ALMA_RB_03#BB_3#SW-01#FULL_RES_2,[J2255-3500_0],[J2255-3500_0],[],"[icrs, 22h55m57.68s, -35d00m00.00s]",9.632156e+10,9.654030e+10
2,gaincaltest2_6,"[CALIBRATE_DELAY#ON_SOURCE, CALIBRATE_PHASE#ON...","(957, 45, 8, 4)","[XX, XY, YX, YY]","[2, 4, 6, 9, 11, 14, 16, 18, 21, 23, 26]",X0000000000#ALMA_RB_03#BB_4#SW-01#FULL_RES_3,[J2255-3500_0],[J2255-3500_0],[],"[icrs, 22h55m57.68s, -35d00m00.00s]",9.819656e+10,9.841530e+10


In [4]:
dataset = sub_ps["gaincaltest2_0"]

In [ ]:
V = sps.VISIBILITY.mean(dim="time").data.compute()

In [ ]:
s = sps.VISIBILITY.shape
s

In [ ]:
def from_vis(dataset):
    shape = dataset.VISIBILITY.shape

    # There should be a gain value for each independent antenna. Here we choose antennai names but either
    # would work fine.
    n_parameters = np.unique(dataset.baseline_antenna1_name).shape[0]
    
    
    identity = np.identity(n_parameters, dtype=np.complex64)

    return np.tile(identity, reps=[*shape, 1, 1])

In [6]:
gain_jones = cv.jones.GainJones()

In [8]:
G = gain_jones.from_visibility(dataset=dataset)

In [10]:
G.matrix.shape

(957, 45, 8, 4, 9, 9)

In [66]:
import numpy as np

test = np.array([
    [
        [
            [1, 0], 
            [0, 1]
        ],
        [
            [2, 0], 
            [0, 2]
        ]
    ],
    [
        [
            [3, 0], 
            [0, 3]
        ],
        [
            [4, 0], 
            [0, 4]
        ]
    ]
])

In [67]:
def diagonal(x):
    n_axis = len(x.shape)
    axis1 = n_axis - 2
    axis2 = n_axis - 1
    
    return test.diagonal(axis1=axis1, axis2=axis2)

In [68]:
diagonal(test)

array([[[1, 1],
        [2, 2]],

       [[3, 3],
        [4, 4]]])

In [35]:
param = np.array([1, 1, 2, 2, 3, 3, 4, 4])

In [64]:
param.reshape((4, 2, 2))

In [41]:
z = np.zeros((4, 2, 2))

In [48]:
I = np.identity(2)

In [62]:
Z = z+I

In [61]:
p = param.reshape(4, 2, 1)

In [63]:
Z*p

array([[[1., 0.],
        [0., 1.]],

       [[2., 0.],
        [0., 2.]],

       [[3., 0.],
        [0., 3.]],

       [[4., 0.],
        [0., 4.]]])